### Calibration températures de Tourterelle en Infrarouge

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys

sys.path.append('../drone_IR')
from open_IRimage_dji import open_IR_image


In [12]:
path2csv = r"F:\bicwin26\Summary\calibration_IR\calibration_Tourterelle_0209.csv"

In [28]:
csvdata = pd.read_csv(path2csv, sep=';')

def convert_to_float(value):
    if type(value) == float:
        return value
    else:
        return float(value.replace(',', '.'))

T_IRcsvdata = np.zeros(len(csvdata['T_IR_celsius'].values))
T_ref_SBE56_num1_celsius = np.zeros(len(csvdata['T_ref_SBE56_num1_celsius'].values))
T_ref_SBE56_num2_celsius = np.zeros(len(csvdata['T_ref_SBE56_num2_celsius'].values))
for i in range(len(csvdata['T_IR_celsius'].values)):
    
    T_IRcsvdata[i] = convert_to_float(csvdata['T_IR_celsius'].values[i])
    T_ref_SBE56_num1_celsius[i] = convert_to_float(csvdata['T_ref_SBE56_num1_celsius'].values[i])
    T_ref_SBE56_num2_celsius[i] = convert_to_float(csvdata['T_ref_SBE56_num2_celsius'].values[i])